In [1]:
import ETCE.ScenarioA.Ex04.weather_data as wd

# DO NOT REMOVE THE ABOVE IMPORT.
# Additionally, you can import any python standard library

This function must accept a weather prediction of the form:

    {"uuid" : "some-uuid",    
    "timestamp": 00000000,
    "temp":5.33,
    "clouds":100.0,
    "wind_speed":2.48,
    "wind_deg":31.0,
    "wind_gust":4.1,
    "rain":1.54
    }
    
Return the predicted power output of the solar farm.

Solar farm Capacity:    50kW
Solar farm gives 0kW output from 9pm to 6am    

Solar Farm output is linear in cloudiness:
Clouds=0% => Solar Farm outputs at 100% Capacity
Clouds=100% => Solar Farm outputs at 10% Capacity

In [6]:
def solar(weather_prediction) -> float: 
    time = datetime.datetime.utcfromtimestamp(weather_prediction["timestamp"]) + datetime.timedelta(hours=2)
    perc = 100
    if not (6 <= time.hour and (time - datetime.timedelta(seconds=1)).hour <= 20):
        perc *= 0
    max_power = 50.0
    min_clouds = 10.0
    max_clouds = 100.0
    perc *= ( max_clouds - (100.0 - min_clouds) * (weather_prediction["clouds"] / 100) )/100.0
    op = max_power * perc/ 100.0
    return op



This function must accept a weather prediction of the form:

    {"uuid" : "some-uuid",
    "timestamp": 00000000,
    "temp":5.33,
    "clouds":100.0,
    "wind_speed":2.48,
    "wind_deg":31.0,
    "wind_gust":4.1,
    "rain":1.54
    }
    
Return the predicted power output of the wind farm.

Wind farm Capacity: 50kW    
Wind Farm output % is linear in wind_speed

Wind Farm cut in speed = 3m/s
Wind Farm rated speed = 11m/s

In [8]:
def wind(weather_prediction) -> float:
    perc = 100
    max_power = 50.0
    cut_in_speed = 3
    rated_speed = 11

    ws = weather_prediction["wind_speed"]
    if ws <= cut_in_speed:
        perc *= 0

    perc *= (min(ws, rated_speed) - cut_in_speed)/(rated_speed - cut_in_speed)
    op = max_power * perc/ 100
    return op



This function should return the solution to the Ex04

Return a tuple with the following structure:

    (({<integer timestamp>: 
     {"power_output":<float (kW)>,
      "energy_accumulated":<float kWh>},
     ...,
     }, <float average power output in %>),
    ({<integer timestamp>: 
     {"power_output":<float (kW)>,
      "energy_accumulated":<float kWh>},
     ...,
     }, <float average power output in %>))

power_output must correspond to the timestamp (result of a call to
solar() and wind()).

energy_accumulated must be total energy accumulated until that
timestamp (assumed to be 0 at the first timestamp)

The first and second nested tuples represent the power output of
the solar and wind farms respectively

In [4]:
def solution():
    weather_predictions = wd.get_predictions()  # Datastructure created in Ex04 solutions  
    
    # Loop over the predictions and compute the power output
    return (({0000:
                  {"power_output":19,
                   "energy_accumulated":0}# ,
              # ...,
              }, 19),
            ({0000:
                  {"power_output":22,
                   "energy_accumulated":0}# ,
              # ...,
              }, 20))

Evaluate your solution:

In [5]:
from driver import evaluate
wd.i = 0
evaluate(solution, solar, wind)

solution() failed to run:


InvalidOperation: [<class 'decimal.ConversionSyntax'>]